In [5]:
!pip install gtts moviepy pillow

from gtts import gTTS
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip
import os
from IPython.display import HTML
from base64 import b64encode

In [6]:
VIDEO_WIDTH, VIDEO_HEIGHT = 1280, 720
# FONT_PATH = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf" # Comment out or remove this line

# User input
topic = input("Enter podcast topic: ")

# Simple local dialogue generator
def generate_dialogue(topic):
    return [
        f"Riya: Welcome to today's podcast! Our topic is {topic}.",
        f"Diya: Yes, it's a fascinating subject with so much to explore today.",
        f"Riya: Let's dive into some interesting points and perspectives.",
        f"Diya: Absolutely, we'll share facts, opinions, and a few surprises.",
        f"Riya: That wraps it up for {topic}, thanks for listening!",
        "Diya: See you next time on our AI podcast!"
    ]

script_lines = generate_dialogue(topic)
# Make folders
os.makedirs("scenes", exist_ok=True)
os.makedirs("audio", exist_ok=True)

# Removed programmatic scene image creation
# Use the paths of the selected files, repeating to match the number of script lines
selected_scene_images = ["/content/scene_0.png","/content/scene_2.png","/content/scene_1.png","/content/scene_4.png", "/content/scene_3.png", "/content/scene_5.png"]
# Ensure there are enough images for all script lines (currently 6)
image_files = (selected_scene_images * (len(script_lines) // len(selected_scene_images) + 1))[:len(script_lines)]

audio_files = []

for i, line in enumerate(script_lines):
    voice_lang = "en"  # Default English
    tts = gTTS(text=line, lang=voice_lang, tld="com" if i % 2 == 0 else "co.uk")
    audio_file = f"audio/line_{i}.mp3"
    tts.save(audio_file)
    audio_files.append(audio_file)

# Create intro and outro
# Removed programmatic intro image creation
# Removed programmatic outro image creation

tts_intro = gTTS(text=f"Welcome to our podcast on {topic}", lang="en", tld="com")
tts_intro_path = "audio/intro.mp3"
tts_intro.save(tts_intro_path)
tts_outro = gTTS(text="Thanks for watching! See you next time.", lang="en", tld="com.au")
tts_outro_path = "audio/outro.mp3"
tts_outro.save(tts_outro_path)

# ---------------- VIDEO ASSEMBLY ----------------
clips = []

# Intro
# Use the user's selected intro image path
intro_image_path = "/content/intro.png" # Use the path of the selected file
intro_clip = ImageClip(intro_image_path).set_duration(3).set_audio(AudioFileClip(tts_intro_path))
clips.append(intro_clip.crossfadein(1))

# Main scenes
for img_path, aud_path in zip(image_files, audio_files):
    scene_clip = ImageClip(img_path).set_duration(AudioFileClip(aud_path).duration + 0.5).set_audio(AudioFileClip(aud_path))
    clips.append(scene_clip.crossfadein(0.5))

# Outro
# Use the user's selected outro image path
outro_image_path = "/content/outro.png" # Use the path of the selected file
outro_clip = ImageClip(outro_image_path).set_duration(3).set_audio(AudioFileClip(tts_outro_path))
clips.append(outro_clip.crossfadein(1))

final_video = concatenate_videoclips(clips, method="compose")
final_video.write_videofile("podcast.mp4", fps=24)

# ---------------- PLAY IN COLAB ----------------
mp4 = open('podcast.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=500 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")

Enter podcast topic: friend
Moviepy - Building video podcast.mp4.
MoviePy - Writing audio in podcastTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video podcast.mp4



Moviepy - Done !
Moviepy - video ready podcast.mp4
